# Data augmentation

Giới hạn nhận diện:
- Biển vuông
- Chỉ nhận diện trên biển đen trắng

In [1]:
import os
import random
import xml.etree.ElementTree as ET
import shutil
import cv2
import matplotlib.pyplot as plt
from utils import *
import albumentations as A
from albumentations.core.transforms_interface import DualTransform
import random
from YOLODetector import YOLODetector

plt.rcParams["figure.figsize"] = (10,10)

## Pick some car images

In [ ]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        #width = int(root.find('size').find('width').text)
        #height = int(root.find('size').find('height').text)

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes


In [ ]:
lst_files = os.listdir("lp_detection/kaggle_car/annotations")
lst_files = [i for i in lst_files if i[-3:] == 'xml']

In [ ]:
anno_path = "lp_detection/kaggle_car/annotations/"
images_path = "lp_detection/kaggle_car/images/"
save_location = 'lp_detection/lp_det_train_data/'

for file in lst_files:
    name, boxes = read_content(anno_path+file)
    
    img = cv2.imread(images_path+name)
    img, boxes = fit_to_square(img, bboxes=boxes)
    cv2.imwrite(save_location+file[:-3]+'jpg', img)
    height, width = img.shape[:2]
    
    with open(save_location+file[:-3]+'txt', 'w') as f:
        for box in boxes:
            x, y = int((box[2] + box[0]) / 2), int((box[3] + box[1]) / 2)
            w, h = box[2] - box[0] + 1, box[3] - box[1] + 1
            yolo_anno = [x/width, y/height, w/width, h/height]
            yolo_anno = [0] + [round(v, 6) for v in yolo_anno]
            for item in yolo_anno:
                f.write("%s " %item)
            f.write('\n')


## Pick some bike images

In [ ]:
lst_files = os.listdir("lp_detection/bike_data")
lst_files = [i for i in lst_files if i[-3:] == 'jpg' and len(i) > 11]

In [ ]:
anno_path = "lp_detection/bike_data/"
images_path = "lp_detection/bike_data/"
save_location = 'lp_detection/lp_det_train_data/'

for file in lst_files:
    img = cv2.imread(images_path+file)
    with open(anno_path+file[:-3]+'txt') as f:
        boxes = [float(num) for num in f.read().split(' ')][1:]
    boxes = yolo_to_bbox(img, boxes)
    boxes = [boxes]
    img, boxes = fit_to_square(img, bboxes=boxes)
    cv2.imwrite(save_location+file, img)
    height, width = img.shape[:2]
    
    with open(save_location+file[:-3]+'txt', 'w') as f:
        box = boxes[0]
        x, y = int((box[2] + box[0]) / 2), int((box[3] + box[1]) / 2)
        w, h = box[2] - box[0] + 1, box[3] - box[1] + 1
        yolo_anno = [x/width, y/height, w/width, h/height]
        yolo_anno = [0] + [round(v, 6) for v in yolo_anno]
        for item in yolo_anno:
            f.write("%s " %item)


## Augmentation

In [ ]:
class AddBorder(DualTransform):
    def __init__(self, low=0, high=150, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        self.low = low
        self.high = high

    def apply(self, img, **params):
        top = random.randint(self.low, self.high)
        bottom = random.randint(self.low, self.high)
        left = random.randint(self.low, self.high)
        right = random.randint(self.low, self.high)

        original_width, original_height = img.shape[1], img.shape[0]
        image = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
        
        self.saved_params = {
            'left': left,
            'top': top,
            'applied_w': image.shape[1],
            'applied_h': image.shape[0],
            'ori_w': original_width,
            'ori_h': original_height,
        }

        image = cv2.resize(image, (original_width, original_height))

        return image

    def apply_to_bbox(self, bbox, **params):
        x_min, y_min, x_max, y_max = bbox[:4]
        x_min, y_min = round(x_min * self.saved_params["ori_w"]), round(y_min * self.saved_params["ori_h"])
        x_max, y_max = round(x_max * self.saved_params["ori_w"]), round(y_max * self.saved_params["ori_h"])

        x_min, y_min = x_min + self.saved_params["left"], y_min + self.saved_params["top"]
        x_max, y_max = x_max + self.saved_params["left"], y_max + self.saved_params["top"]

        x_min, y_min = x_min / self.saved_params["applied_w"], y_min / self.saved_params["applied_h"]
        x_max, y_max = x_max / self.saved_params["applied_w"], y_max / self.saved_params["applied_h"]

        return (x_min, y_min, x_max, y_max)

In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    AddBorder(low=0, high=200, p=0.5),
    A.RandomResizedCrop(416, 416, scale=(0.005,0.3), ratio=(0.75,1.33), interpolation=cv2.INTER_NEAREST, p=0.5),
    A.Rotate(limit=10, border_mode=cv2.BORDER_CONSTANT, value=0, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.9))

In [ ]:
lst_files = os.listdir('lp_detection/lp_det_train_data')
lst_files = [i for i in lst_files if i[-3:] == 'txt']
len(lst_files)

2173

In [ ]:
original_path = 'lp_detection/lp_det_train_data/'
save_location = 'lp_detection/aug/'

selected_files = random.choices(lst_files, k=1000)
file_count = 0
for file in selected_files:
    try:
        im_path = original_path+file[:-3]+'jpg'
        img = cv2.imread(im_path)

        boxes = []
        with open(original_path+file) as f:
            for line in f.readlines():
                boxes.append([float(num) for num in line.split(' ')[:-1]][1:] + ['license_plate'])

        while True:
            transformed = transform(image=img, bboxes=boxes)
            if transformed['bboxes'] != []:
                break
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']

        cv2.imwrite(save_location+'aug'+str(file_count)+'.jpg', transformed_image)
        yolo_anno = [[0] + list(line[:4]) for line in transformed_bboxes]
        with open(save_location+'aug'+str(file_count)+'.txt', 'w') as f:
            for line in yolo_anno:
                for item in line:
                    f.write("%s " %round(item, 6))
                f.write('\n')
        file_count += 1
    except:
        continue
    #break

# Train

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Sep 29 16:17:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
cd drive/MyDrive/Colab\ Notebooks/darknet

/content/drive/MyDrive/Colab Notebooks/darknet


In [7]:
!unzip lp_det_cfg.zip

Streaming output truncated to the last 5000 lines.
  inflating: lp_det_cfg/data/0200_08054_b.txt  
  inflating: lp_det_cfg/data/0356_05992_b.txt  
  inflating: lp_det_cfg/data/aug960.jpg  
  inflating: lp_det_cfg/data/0124_06079_b.txt  
  inflating: lp_det_cfg/data/0235_06933_b.txt  
  inflating: lp_det_cfg/data/0146_01335_b.jpg  
  inflating: lp_det_cfg/data/0209_08290_b.jpg  
  inflating: lp_det_cfg/data/0251_01938_b.jpg  
  inflating: lp_det_cfg/data/aug790.jpg  
  inflating: lp_det_cfg/data/0131_00942_b.jpg  
  inflating: lp_det_cfg/data/aug948.jpg  
  inflating: lp_det_cfg/data/aug784.jpg  
  inflating: lp_det_cfg/data/0439_06768_b.txt  
  inflating: lp_det_cfg/data/0238_01984_b.jpg  
  inflating: lp_det_cfg/data/0425_04108_b.jpg  
  inflating: lp_det_cfg/data/aug651.txt  
  inflating: lp_det_cfg/data/aug889.txt  
  inflating: lp_det_cfg/data/aug137.txt  
  inflating: lp_det_cfg/data/0248_01979_b.jpg  
  inflating: lp_det_cfg/data/0131_00774_b.jpg  
  inflating: lp_det_cfg/data/01

In [6]:
#Tạo file train.txt, valid.txt
import os
import numpy as np

lst_files = os.listdir("lp_det_cfg/data/")
lst_images = [file for file in lst_files if file[-3:] == 'jpg']    

#Tách ảnh ra làm tập validation  
val_size = int(len(lst_images) * 0.3)
print(val_size)
random_idx = np.random.choice(len(lst_images), size=val_size, replace=False)

#Tạo file train.txt
with open("lp_det_cfg/train.txt","w") as f:
  for idx in range(len(lst_images)):
    if idx not in random_idx:
      f.write("lp_det_cfg/data/"+lst_images[idx]+"\n")
      
#Tạo file valid.txt
with open("lp_det_cfg/valid.txt","w") as f:
    for idx in random_idx:
      f.write("lp_det_cfg/data/"+lst_images[idx]+"\n")

822


In [8]:
!chmod +x ./darknet

In [9]:
!./darknet detector train lp_det_cfg/obj.data lp_det_cfg/yolov4-tiny-lp-det.cfg yolov4-tiny.conv.29 -map \
-dont_show > lp_det_cfg/yolov4-tiny-lp-det.log

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.878103), count: 3, class_loss = 0.015000, iou_loss = 0.815710, total_loss = 0.830710 
 total_bbox = 216137, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.831400), count: 3, class_loss = 0.034112, iou_loss = 0.224426, total_loss = 0.258539 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.844718), count: 2, class_loss = 0.006884, iou_loss = 0.711867, total_loss = 0.718751 
 total_bbox = 216142, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.889920), count: 2, class_loss = 0.000010, iou_loss = 0.393234, total_loss = 0.393244 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.901536), count: 4, class_loss = 0.003159, iou_loss = 1.238587, total_loss = 1.241746 
 total_

# Test

In [2]:
lp_det = YOLODetector('lp_detection/yolov4-tiny-lp-det_best.weights', 'lp_detection/yolov4-tiny-lp-det.cfg')

In [ ]:
img = cv2.imread("web_server/test.jpg")
img = fit_to_square(img, size=1000)

bboxes_yolo = lp_det.detect(img)
bboxes = [yolo_to_bbox(img, box) for box in bboxes_yolo]

for bbox in bboxes:
    color = (0,255,255)
    draw_bbox(img, "LP", bbox, color, 2)
cv2.imwrite('result.jpg', img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))